# SNAKEMAKE

In [173]:
%%writefile snakefile2
from pathlib import Path

## This is the new improved pipe for smallseq analysis and filtering. 

#TODO: CHANGE TO ADD CONFIG FILE AS INPUT! 

# CHANGE ME
READS = [file.stem for file in Path('SRR_new/').iterdir() if file.suffix == '.fastq']

rule all:
    input:
        expand('snakemake_counts/idxstats/mature/{file}_counts_mature.tsv', file = READS),
        expand('snakemake_bowtie2/mature/{file}_mature_bowtie2_aligned.bam', file = READS),
        expand('snakemake_salmon/mature/{file}_salmon_mature/', file = READS),
        expand('snakemake_salmon/transcriptome/{file}_salmon_transcriptome/', file = READS),
        expand('snakemake_bowtie2/mature_T/{file}_mature_bowtie2_aligned.bam', file = READS),
        'snakemake_counts/featurecounts/genome/featurecounts_genome_clean.txt'
        
        
### MIRBASE ALIGNMENT ###:         
        
rule remove_umi:
    input:
        raw_reads = 'SRR_new/{file}.fastq'
    output:
        umi_reads = 'snakemake_umi/umi_trimmed/{file}_umi.fastq',
        log_file = 'snakemake_umi/reports/{file}_umi.log'
    shell:
        "umi_tools extract --extract-method=regex --bc-pattern='(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA)' " 
        "-I {input.raw_reads} -S {output.umi_reads} -L {output.log_file}" 
        #FOR REMOVING 3' ADAPTER: '(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA).*(?P<discard_2>TGGAATTCT.+)' 
        
# REMOVED -u here because it filters the 5' ends of the read. I have already done this in umi_tools! 
rule cutadapt:
    input:
        'snakemake_umi/umi_trimmed/{file}_umi.fastq'
    output:
        cutadapt_reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq',
        log = 'snakemake_cutadapt/reports/{file}_cutadapt.log'
    params:
        adapter_file = 'adapters/adapters.txt'
    shell:
        "cutadapt -a file:{params.adapter_file} -e 0.1 -O 1 --quiet --minimum-length 18 " 
        "--maximum-length 40 -o {output.cutadapt_reads} {input} > {output.log}"

# maybe change the --norc to see if the alignment rate goes up? 
rule bowtie1_mature_alignment:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        aligned_sam = 'snakemake_bowtie/mature/aligned/{file}_bowtie_aligned.sam',
        non_aligned = 'snakemake_bowtie/mature/not_aligned/{file}_bowtie_non_aligned.fastq',
        log = 'snakemake_bowtie/mature/reports/{file}_bowtie_aligned.log'
    params:
        index = 'indexes/bowtie_index/hsa_mature_index/hsa_mature'
    shell:
        "bowtie -n 0 -l 32 --norc --best --strata -m 1 -x {params.index} {input.reads} "
        "--un {output.non_aligned} -S {output.aligned_sam} 2> {output.log}" 
        
rule mature_sam_to_bam:
    input:
        aligned_sam = 'snakemake_bowtie/mature/aligned/{file}_bowtie_aligned.sam'
    output:
        sorted_bam = 'snakemake_bowtie/mature/aligned/{file}_bowtie_aligned.sorted.bam'
    shell:
        "samtools sort -o {output.sorted_bam} {input.aligned_sam} && rm {input.aligned_sam} "
        "&& samtools index {output.sorted_bam}"
            
rule mirbase_hairpin_alignment:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        aligned_sam = 'snakemake_bowtie/hairpin/aligned/{file}_bowtie_aligned.sam',
        non_aligned = 'snakemake_bowtie/hairpin/not_aligned/{file}_bowtie_non_aligned.fastq',
        log = 'snakemake_bowtie/hairpin/reports/{file}_bowtie_aligned.log'
    params:
        index = 'indexes/bowtie_index/hsa_hairpin_index/hsa_hairpin'
    shell:
        "bowtie -n 0 -l 32 --norc --best --strata -m 1 -x {params.index} {input.reads} "
        "--un {output.non_aligned} -S {output.aligned_sam} 2> {output.log}" 

rule hairpin_sam_to_bam:
    input:
        aligned_sam = 'snakemake_bowtie/hairpin/aligned/{file}_bowtie_aligned.sam'
    output:
        sorted_bam = 'snakemake_bowtie/hairpin/aligned/{file}_bowtie_aligned.sorted.bam'
    shell:
        "samtools sort -o {output.sorted_bam} {input.aligned_sam} && rm {input.aligned_sam} "
        "&& samtools index {output.sorted_bam}"

# Just testing STAR. Really low alignment rate, doesnt align at all.
rule star_all_reads:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        'snakemake_star/{file}_Aligned.out.sam',
        'snakemake_star/{file}_Log.final.out',
        'snakemake_star/{file}_Log.out',
        'snakemake_star/{file}_Log.progress.out',
        'snakemake_star/{file}_SJ.out.tab'
    params:
        index = 'indexes/star_index/',
        prefix = 'snakemake_star/{file}_'
    shell:
        "STAR --runThreadN 4 --genomeDir {params.index} --readFilesIn {input.reads} " 
        "--readFilesCommand zcat --outSAMstrandField intronMotif "
        "--outFilterMultimapNmax 50 --outFilterScoreMinOverLread 0 --outFilterMatchNmin 18 " 
        "--outFilterMatchNminOverLread 0 --outFilterMismatchNoverLmax 0.04 --alignIntronMax 1 --outFileNamePrefix {params.prefix}"

rule star_sam_to_bam:
    input:
        aligned_sam = 'snakemake_star/{file}_Aligned.out.sam'
    output:
        sorted_bam = 'snakemake_star/{file}_aligned.sorted.bam'
    shell:
        "samtools sort -o {output.sorted_bam} {input.aligned_sam} && rm {input.aligned_sam} "
        "&& samtools index {output.sorted_bam}"
        
rule deduplicate_umi_mature:
    input:
        'snakemake_bowtie/mature/aligned/{file}_bowtie_aligned.sorted.bam' 
    output:
        deduplicated = 'snakemake_deduplicated/mature/reads/{file}_deduplicated_mature.bam',
        log = 'snakemake_deduplicated/mature/reports/{file}_deduplicated_mature.log'
    shell:
        "umi_tools dedup --method=unique -I {input} -S {output.deduplicated} -L {output.log} && "
        "samtools index {output.deduplicated}"
        
# count reads with samtools idxstats from the deduplicated files from the alignment to mirbase mature      
rule samtools_count_idxstats_mirbase_mature:
    input:
        deduplicated = 'snakemake_deduplicated/mature/reads/{file}_deduplicated_mature.bam'
    output:
        idxstats_count = 'snakemake_counts/idxstats/mature/{file}_counts_mature.tsv'
    shell:
        "samtools idxstats {input.deduplicated} | cut -f 1,3 > {output.idxstats_count}" 
    
    
### Align unalign reads that did not align to mirBase to genome ###


#Align better than star, around 30-40% alignment. Looser paramters than bowtie1 and to the mirbase mature. 
rule bowtie2_genome_alignment:
    input:
        non_aligned = 'snakemake_bowtie/mature/not_aligned/{file}_bowtie_non_aligned.fastq'
    output:
        aligned = 'snakemake_bowtie2/mirbase_unaligned/{file}_aligned_bowtie2.bam',
        log = 'snakemake_bowtie2/mirbase_unaligned/reports/{file}.log'
    params:
        index = 'indexes/bowtie2_index/genome/GRCh38'
    shell:
        "bowtie2 -k 100 --local --very-sensitive-local -x {params.index} -U {input.non_aligned} 2> {output.log} | " 
        "samtools view -b - | samtools sort -o {output.aligned} - && samtools index {output.aligned}"
        
# deduplicate umi from the reads aligned to genome with bowtie2     
rule deduplicate_umi_bowtie2_alignment:
    input:
        aligned = 'snakemake_bowtie2/mirbase_unaligned/{file}_aligned_bowtie2.bam' 
    output:
        deduplicated = 'snakemake_deduplicated/genome/reads/{file}_deduplicated_genome.bam',
        log = 'snakemake_deduplicated/genome/reports/{file}_deduplicated_genome.log'
    shell:
        "umi_tools dedup --method=unique -I {input} -S {output.deduplicated} -L {output.log} && "
        "samtools index {output.deduplicated}"

# count the deduplicated reads aligned to genome with featureCounts 
rule count_reads_genome_featurecounts:
    input:
        gtf_file = 'genomes/gencode.v38.chr_patch_hapl_scaff.annotation.gtf'
    params:
        reads_list = 'snakemake_deduplicated/genome/reads/*.bam'
    output:
        counts = 'snakemake_counts/featurecounts/genome/featurecounts_genome.txt' 
    shell:
        "featureCounts -a {input.gtf_file} -o {output.counts} {params.reads_list}"

# clean columns in featurecounts matrix 
rule clean_featurecounts:
    input:
        'snakemake_counts/featurecounts/genome/featurecounts_genome.txt'
    output:
        'snakemake_counts/featurecounts/genome/featurecounts_genome_clean.txt'
    script:
        'scripts/clean_featurecounts.py'

        
# Testing different aligners and alignments

# Did not align AT ALL!! 
rule bowtie2_mature_alignment:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        aligned = 'snakemake_bowtie2/mature/{file}_mature_bowtie2_aligned.bam',
        log = 'snakemake_bowtie2/mature/reports/{file}.log'
    params:
        index = 'indexes/bowtie2_index/mature/hsa_mature'
    shell:
        "bowtie2 -k 100 --local --very-sensitive-local -x {params.index} -U {input.reads} 2> {output.log} | " 
        "samtools view -b - | samtools sort -o {output.aligned} - && samtools index {output.aligned}"

# trying salmon to mature.
# maybe add --seqbias, --useVBOpt and --numBootStraps?
rule salmon_mature_mirbase:
    input:
        'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        directory('snakemake_salmon/mature/{file}_salmon_mature/')
    params:
        index = 'indexes/salmon_index/mature/hsa_mature/'
    shell:
        """
        salmon quant -l A -i {params.index} -r {input} -o {output} --writeUnmappedNames -q
        """
        
# maybe add --seqbias, --useVBOpt(DEFAULT!) and --numBootStraps?        
rule salmon_transcriptome:
    input:
        'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        directory('snakemake_salmon/transcriptome/{file}_salmon_transcriptome/')
    params:
        index = 'indexes/salmon_index/transcriptome/transcriptome/'
    shell:
        """
        salmon quant -l A --seqBias -i {params.index} -r {input} -o {output} --writeUnmappedNames -q
        """

#testing bowtie2 against mirbase with T 
rule bowtie2_mature_T_alignment:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        aligned = 'snakemake_bowtie2/mature_T/{file}_mature_bowtie2_aligned.bam',
        log = 'snakemake_bowtie2/mature_T/reports/{file}.log'
    params:
        index = 'indexes/bowtie2_index/mature_T/hsa_mature_T'
    shell:
        "bowtie2 -k 100 --local --very-sensitive-local -x {params.index} -U {input.reads} 2> {output.log} | " 
        "samtools view -b - | samtools sort -o {output.aligned} - && samtools index {output.aligned}"
        

Overwriting snakefile2


In [5]:
%%writefile Snakefile
import subprocess
from pathlib import Path


# CHANGE ME
READS = [str(file.stem).split('_')[0] for file in Path('snakemake_cutadapt/').iterdir()]


rule all:
    input:
        expand('snakemake_star/{file}_aligned.sorted.bam', file = READS)
        
    
rule remove_umi:
    input:
        raw_reads = 'test_data/{file}.gz'
    output:
        umi_reads = 'snakemake_umi/umi_trimmed/{file}_umi.gz',
        log_file = 'snakemake_umi/reports/{file}_umi.log'
    shell:
        """
        umi_tools extract -p NNNNNNNN -I {input.raw_reads} -S {output.umi_reads} -L {output.log_file}
        """

rule cutadapt:
    input:
        'snakemake_umi/umi_trimmed/{file}_umi.gz'
    output:
        cutadapt_reads = 'snakemake_cutadapt/{file}_umi_cutadapt.gz'
    params:
        adapter_file = 'adapters/adapters.txt'
    shell:
        """
        cutadapt -a file:{params.adapter_file} -e 0.1 -O 1 -u 2 --quiet --minimum-length 18 -o {output.cutadapt_reads} {input}
        """
    
# settings from Ziemann et al 2016: bowtie2 -k 100 --local --very-sensitive-local -x ref -U read
# NOTE TO SELF: added samtools index here without testing
# COMMENT: why not just use setting which dont allow soft clipping?? like end-to-end alignment???
rule bowtie2_and_samtools:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.gz'
    output:
        aligned_bam = 'snakemake_bowtie2/aligned/{file}_bowtie2_aligned.sorted.bam',
        non_aligned = 'snakemake_bowtie2/not_aligned/{file}_bowtie2_non_aligned.gz',
        log = 'snakemake_bowtie2/aligned/{file}_bowtie_log.txt'
    params:
        index = 'indexes/bowtie2_index/GRCh38'
    shell:
        """
        bowtie2 -x {params.index} -U {input.reads} --un-gz {output.non_aligned} 2> {output.log} | samtools view -b - | samtools sort -o {output.aligned_bam} - && samtools index {output.aligned_bam} 
        """

rule star:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.gz'
    output:
        'snakemake_star/{file}_Aligned.out.sam',
        'snakemake_star/{file}_Log.final.out',
        'snakemake_star/{file}_Log.out',
        'snakemake_star/{file}_Log.progress.out',
        'snakemake_star/{file}_SJ.out.tab'
    params:
        index = 'indexes/star_index/',
        prefix = 'snakemake_star/{file}_'
    shell:
        "STAR --runThreadN 4 --genomeDir {params.index} --readFilesIn {input.reads} --readFilesCommand zcat --outSAMstrandField intronMotif "
        "--outFilterMultimapNmax 50 --outFilterScoreMinOverLread 0 --outFilterMatchNmin 18 --outFilterMatchNminOverLread 0 --outFilterMismatchNoverLmax 0.04 --alignIntronMax 1 --outFileNamePrefix {params.prefix}"

# SHOUlD output sorted bam file from star instead!!!!
rule sam_to_bam:
    input:
        sam = 'snakemake_star/{file}_Aligned.out.sam'
    output:
        sorted_bam = 'snakemake_star/{file}_aligned.sorted.bam'
    shell:
        """
        samtools view -bS {input.sam} | samtools sort -o {output.sorted_bam} - && samtools index {output.sorted_bam}
        """

#add script here 
rule remove_softclip:
    input:
        'snakemake_star/{file}_aligned.sorted.bam' #maybe input sam? 
    output:
        'removed_clipped/{file}_clipped.bam'
    script:
        'scripts/XXXX.py'


rule subsample_bam_readlength:
    input:
        'snakemake_star/{file}_aligned.sorted.bam'
    output:
        'snakemake_subsampled/{file}_subsampled.bam'
    params:
        max_cutoff = 40 
        
    shell:
        """
        samtools view -h {input} | awk \'length($10) <= {params.max_cutoff} || $1~"@"\' | samtools view -bS - > {output}
        """

rule umi_deduplication:
    input:
        'snakemake_subsampled/{file}_subsampled.bam'
    output:
        deduplicated = 'snakemake_removed_umi/{file}_removedumi.bam',
        log = 'snakemake_removed_umi/{file}_log.txt'
    params:
        output_stats = {file}
    shell:
        """
        umi_tools dedup --method adjacency --output-stats={params.output_stats} --further-stats --read-length -I {input} -S {output.deduplicated} -L {output.log}
        """
#rule index_bam:
#rule merge_bam:
#rule fastqc:
#rule multiqc:  


Overwriting Snakefile


# SCRIPTS

In [26]:
%%writefile scripts/download_SRA.py
import pandas as pd
import subprocess

sra = pd.read_csv('sra_table.txt')

for file in sra.Run:
    command = f"prefetch {file}".split()
    subprocess.call(command)



Overwriting scripts/download_SRA.py


# TESTCODE

In [29]:
import pandas as pd
from janitor import clean_names
from datar.all import *

sra = pd.read_csv('test_data/sra_table.txt').clean_names() >> select(~f.sample_type)

sra >> filter(f.cell_type == 'GBM')


,run,assay_type,avgspotlen,bases,bioproject,biosample,bytes,center_name,consent,datastore_filetype,...,librarylayout,libraryselection,librarysource,organism,platform,releasedate,sample_name,source_name,sra_study,cell_type
,<object>,<object>,<int64>,<int64>,<object>,<object>,<int64>,<object>,<object>,<object>,...,<object>,<object>,<object>,<object>,<object>,<object>,<object>,<object>,<object>,<object>
0,SRR3495523,ncRNA-Seq,43,110906503,PRJNA321192,SAMN04976967,112821441,GEO,public,"fastq,sra",...,SINGLE,size fractionation,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2016-10-17T00:00:00Z,GSM2149376,Glioblastoma,SRP074776,GBM
1,SRR3495524,ncRNA-Seq,43,108781615,PRJNA321192,SAMN04976968,109873177,GEO,public,"fastq,sra",...,SINGLE,size fractionation,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2016-10-17T00:00:00Z,GSM2149377,Glioblastoma,SRP074776,GBM
2,SRR3495525,ncRNA-Seq,43,149699383,PRJNA321192,SAMN04976969,150617486,GEO,public,"fastq,sra",...,SINGLE,size fractionation,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2016-10-17T00:00:00Z,GSM2149378,Glioblastoma,SRP074776,GBM
3,SRR3495526,ncRNA-Seq,43,157402618,PRJNA321192,SAMN04976970,158483005,GEO,public,"fastq,sra",...,SINGLE,size fractionation,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2016-10-17T00:00:00Z,GSM2149379,Glioblastoma,SRP074776,GBM
4,SRR3495527,ncRNA-Seq,43,129724206,PRJNA321192,SAMN04976971,131526400,GEO,public,"fastq,sra",...,SINGLE,size fractionation,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2016-10-17T00:00:00Z,GSM2149380,Glioblastoma,SRP074776,GBM
5,SRR3495528,ncRNA-Seq,43,133369359,PRJNA321192,SAMN04976972,135118332,GEO,public,"fastq,sra",...,SINGLE,size fractionation,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2016-10-17T00:00:00Z,GSM2149381,Glioblastoma,SRP074776,GBM


In [50]:
from pathlib import Path

files = [file for file in Path('snakemake_cutadapt/').iterdir()]

for file in files:
    new = str(file).replace('.fastq', '')
    file.rename(new)

# SNAKEMAKE SCRIPTS

In [1]:
%%writefile scripts/remove_soft_clip.py

# TODO: Fix this to insert snakemake.inputs and outputs. PRobably dont have to loop through, since it will pipe one sample each go. 


import pysam
from pathlib import Path


def remove_reads(in_bam_file):
    in_bam = pysam.AlignmentFile(in_bam_file)
    path_out_bam = Path(out_dir, (in_bam_file.stem + 'clipped.bam'))
    out_bam = pysam.AlignmentFile(path_out_bam, 'wb', template=in_bam)
    total_reads = 0
    reads_after_clipped = 0
    logfile = Path(out_dir, 'logfile.txt')
    
    #limits
    allowed_num_clipped_bases_5p = 0 #snakemake.params.variable ?? 
    allowed_num_clipped_bases_3p = 3
        
    for read in in_bam:
        total_reads += 1
        read_name = read.qname
        cigar = read.cigar  #this is a list of tuples;  example: cigar=[(0, 25), (4, 8)]
        
        num_softclipped_5p = 0
        num_softclipped_3p = 0
        num_hardclipped = 0
        num_insertions = 0
        num_deletions = 0
        
        for i,e in enumerate(cigar):
            if e[0] == 5:
                num_hardclipped += e[1]
            elif e[0] == 1:
                num_insertions += e[1]
            elif e[0] == 2:
                num_deletions += e[1]
            elif i == 0 and e[0] == 4:
                num_softclipped_5p += e[1]
            elif i == len(cigar)-1 and e[0] == 4:
                num_softclipped_3p += e[1]

        if num_softclipped_5p > allowed_num_clipped_bases_5p:
            continue
        elif num_softclipped_3p > allowed_num_clipped_bases_3p:
            continue
        elif num_hardclipped > 0: 
            continue
        elif num_insertions > 0: 
            continue 
        elif num_deletions > 0: 
            continue   
        
        out_bam.write(read)
        reads_after_clipped += 1
        
    out_bam.close()
    
    pysam.sort('-o', str(path_out_bam), str(path_out_bam))  
    pysam.index(str(path_out_bam))
    
    with open(logfile, 'a') as f:
        print(f" Procent removed in {read_name}: {100 * (1- reads_after_clipped / total_reads): .2f}%", file=f)


        
#directory making
out_dir = Path('clipped_removed/') #snakemake.output
if not out_dir.exists():
    out_dir.mkdir(parents=True, exist_ok=True)

    
files = [file for file in Path('snakemake_star/').iterdir() if file.suffix == '.bam'] #snakemake.input
        
#main loop
for bamfile in files:
    print(f'started {bamfile}')
    remove_reads(bamfile)
    print(f'done {bamfile}')
  

Overwriting scripts/remove_soft_clip.py


In [141]:
%%writefile scripts/clean_featurecounts.py
import pandas as pd
from datar.all import select, contains
import re
import os


# read in data 
in_file = snakemake.input[0]
out_file = snakemake.output[0]
counts = pd.read_csv(in_file, delimiter='\t', skiprows=1)

# filter for correct columns
counts = (counts >>
 select(contains('Geneid') | contains('bam'))
)

# extract SRR number from the columns
pattern = re.compile(r'SRR\d+')
new_names = ["".join(re.findall(pattern, column)) if not column == 'Geneid' else column for column in counts.columns]

# rename columns with SRR number and save file 
counts.columns = new_names
counts.to_csv(out_file)

# remove original file
os.remove(in_file)



Overwriting scripts/clean_featurecounts.py


In [174]:
%%writefile snakefile_star_debo

## new snake file to only run star to show debo


READS = [file.stem for file in Path('SRR_new/').iterdir() if file.suffix == '.fastq']

rule all:
    input:
        expand('snakemake_star/{file}_aligned.sorted.bam', file = READS)
        
    
rule remove_umi:
    input:
        raw_reads = 'SRR_new/{file}.fastq'
    output:
        umi_reads = 'snakemake_umi/umi_trimmed/{file}_umi.fastq',
        log_file = 'snakemake_umi/reports/{file}_umi.log'
    shell:
        "umi_tools extract --extract-method=regex --bc-pattern='(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA)' " 
        "-I {input.raw_reads} -S {output.umi_reads} -L {output.log_file}" 
        #FOR REMOVING 3' ADAPTER: '(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA).*(?P<discard_2>TGGAATTCT.+)' 
        
# REMOVED -u here because it filters the 5' ends of the read. I have already done this in umi_tools! 
rule cutadapt:
    input:
        'snakemake_umi/umi_trimmed/{file}_umi.fastq'
    output:
        cutadapt_reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq',
        log = 'snakemake_cutadapt/reports/{file}_cutadapt.log'
    params:
        adapter_file = 'adapters/adapters.txt'
    shell:
        "cutadapt -a file:{params.adapter_file} -e 0.1 -O 1 --quiet --minimum-length 18 " 
        "--maximum-length 40 -o {output.cutadapt_reads} {input} > {output.log}"

rule star_all_reads:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        'snakemake_star/{file}_Aligned.out.sam',
        'snakemake_star/{file}_Log.final.out',
        'snakemake_star/{file}_Log.out',
        'snakemake_star/{file}_Log.progress.out',
        'snakemake_star/{file}_SJ.out.tab'
    params:
        index = 'indexes/star_index/',
        prefix = 'snakemake_star/{file}_'
    shell:
        "STAR --runThreadN 4 --genomeDir {params.index} --readFilesIn {input.reads} " 
        "--readFilesCommand zcat --outSAMstrandField intronMotif "
        "--outFilterMultimapNmax 50 --outFilterScoreMinOverLread 0 --outFilterMatchNmin 18 " 
        "--outFilterMatchNminOverLread 0 --outFilterMismatchNoverLmax 0.04 --alignIntronMax 1 --outFileNamePrefix {params.prefix}"

rule star_sam_to_bam:
    input:
        aligned_sam = 'snakemake_star/{file}_Aligned.out.sam'
    output:
        sorted_bam = 'snakemake_star/{file}_aligned.sorted.bam'
    shell:
        "samtools sort -o {output.sorted_bam} {input.aligned_sam} && rm {input.aligned_sam} "
        "&& samtools index {output.sorted_bam}"

Writing snakefile_star_debo


In [179]:
%%writefile snakefile_star_debo_mildare_parametrar

## new snake file to only run star to show debo, changed the cutadapt parameters to include a broader range of reads. 
## This did NOT align at all! 


READS = [file.stem for file in Path('SRR_new/').iterdir() if file.suffix == '.fastq']

rule all:
    input:
        expand('snakemake_star/{file}_aligned.sorted.bam', file = READS)
        
    
rule remove_umi:
    input:
        raw_reads = 'SRR_new/{file}.fastq'
    output:
        umi_reads = 'snakemake_umi/umi_trimmed/{file}_umi.fastq',
        log_file = 'snakemake_umi/reports/{file}_umi.log'
    shell:
        "umi_tools extract --extract-method=regex --bc-pattern='(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA)' " 
        "-I {input.raw_reads} -S {output.umi_reads} -L {output.log_file}" 
        #FOR REMOVING 3' ADAPTER: '(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA).*(?P<discard_2>TGGAATTCT.+)' 
        
# REMOVED -u here because it filters the 5' ends of the read. I have already done this in umi_tools! 
rule cutadapt:
    input:
        'snakemake_umi/umi_trimmed/{file}_umi.fastq'
    output:
        cutadapt_reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq',
        log = 'snakemake_cutadapt/reports/{file}_cutadapt.log'
    params:
        adapter_file = 'adapters/adapters.txt'
    shell:
        "cutadapt -a file:{params.adapter_file} -e 0.1 -O 1 --quiet --minimum-length 12 " 
        "--maximum-length 50 -o {output.cutadapt_reads} {input} > {output.log}"

rule star_all_reads:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        'snakemake_star/{file}_Aligned.out.sam',
        'snakemake_star/{file}_Log.final.out',
        'snakemake_star/{file}_Log.out',
        'snakemake_star/{file}_Log.progress.out',
        'snakemake_star/{file}_SJ.out.tab'
    params:
        index = 'indexes/star_index_ensembl/',
        prefix = 'snakemake_star/{file}_'
    shell:
        "STAR --runThreadN 4 --genomeDir {params.index} --readFilesIn {input.reads} " 
        "--readFilesCommand zcat --outSAMstrandField intronMotif "
        "--outFilterMultimapNmax 50 --outFilterScoreMinOverLread 0 --outFilterMatchNmin 12 " 
        "--outFilterMatchNminOverLread 0 --outFilterMismatchNoverLmax 0.04 --alignIntronMax 1 --outFileNamePrefix {params.prefix}"

rule star_sam_to_bam:
    input:
        aligned_sam = 'snakemake_star/{file}_Aligned.out.sam'
    output:
        sorted_bam = 'snakemake_star/{file}_aligned.sorted.bam'
    shell:
        "samtools sort -o {output.sorted_bam} {input.aligned_sam} && rm {input.aligned_sam} "
        "&& samtools index {output.sorted_bam}"

Overwriting snakefile_star_debo_mildare_parametrar


In [191]:
%%writefile snakefile_different_STAR_parameters

## STAR parameters from ENCODE miRNA-seq read alignment using STAR aligner,
#https://www.encodeproject.org/documents/b4ec4567-ac4e-4812-b2bd-e1d2df746966/@@download/attachment/ENCODE_miRNA-seq_STAR_parameters_v2.pdf

READS = [file.stem for file in Path('SUBSET_DATA/').iterdir() if file.suffix == '.fastq']

rule all:
    input:
        expand('snakemake_star/{file}_Aligned.sortedByCoord.out.bam', file = READS)
        
    
rule remove_umi:
    input:
        raw_reads = 'SUBSET_DATA/{file}.fastq'
    output:
        umi_reads = 'snakemake_umi/umi_trimmed/{file}_umi.fastq',
        log_file = 'snakemake_umi/reports/{file}_umi.log'
    shell:
        "umi_tools extract --extract-method=regex --bc-pattern='(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA)' " 
        "-I {input.raw_reads} -S {output.umi_reads} -L {output.log_file}" 
        #FOR REMOVING 3' ADAPTER: '(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA).*(?P<discard_2>TGGAATTCT.+)' 
        
# REMOVED -u here because it filters the 5' ends of the read. I have already done this in umi_tools! 
rule cutadapt:
    input:
        'snakemake_umi/umi_trimmed/{file}_umi.fastq'
    output:
        cutadapt_reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq',
        log = 'snakemake_cutadapt/reports/{file}_cutadapt.log'
    params:
        adapter_file = 'adapters/adapters.txt'
    shell:
        "cutadapt -a file:{params.adapter_file} -e 0.1 -O 1 --quiet --minimum-length 18 " 
        "--maximum-length 40 -o {output.cutadapt_reads} {input} > {output.log}"

rule star_all_reads:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq',
        gtf = 'genomes/gencode_miRNA_subset.gtf'
    output:
        'snakemake_star/{file}_Aligned.sortedByCoord.out.bam'
    params:
        index = 'indexes/star_index_gencode/',
        prefix = 'snakemake_star/{file}_'
    shell:
        "STAR --runThreadN 4 "
        "--genomeDir {params.index} --readFilesIn {input.reads} "
        "--sjdbGTFfile {input.gtf} "
        "--outFileNamePrefix {params.prefix} "
        "--alignEndsType EndToEnd "
        "--outFilterMismatchNmax 1 "
        "--outFilterMultimapScoreRange 0 "
        "--quantMode TranscriptomeSAM GeneCounts "
        "--outReadsUnmapped Fastx "
        "--outSAMtype BAM SortedByCoordinate "
        "--outFilterMultimapNmax 10 "
        "--outSAMunmapped Within "
        "--outFilterScoreMinOverLread 0 "
        "--outFilterMatchNmin 16 "
        "--alignSJDBoverhangMin 1000 "
        "--alignIntronMax 1 "
        "--outWigType wiggle "
        "--outWigStrand Stranded "
        "--outWigNorm RPM "


    



Overwriting snakefile_different_STAR_parameters


In [195]:
%%writefile snakefile_bowtie1_different_parameters

## Testing Alignment with bowtie with parameters found in 
# https://www.biostars.org/p/360709/
# This aligns well (73% to the genome), however, should make it a little bit more stringent!! 

READS = [file.stem for file in Path('SUBSET_DATA/').iterdir() if file.suffix == '.fastq']

rule all:
    input:
        expand('snakemake_bowtie/aligned/{file}_bowtie_aligned.sam', file = READS)
        
    
rule remove_umi:
    input:
        raw_reads = 'SUBSET_DATA/{file}.fastq'
    output:
        umi_reads = 'snakemake_umi/umi_trimmed/{file}_umi.fastq',
        log_file = 'snakemake_umi/reports/{file}_umi.log'
    shell:
        "umi_tools extract --extract-method=regex --bc-pattern='(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA)' " 
        "-I {input.raw_reads} -S {output.umi_reads} -L {output.log_file}" 
        #FOR REMOVING 3' ADAPTER: '(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA).*(?P<discard_2>TGGAATTCT.+)' 
        
rule cutadapt:
    input:
        'snakemake_umi/umi_trimmed/{file}_umi.fastq'
    output:
        cutadapt_reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq',
        log = 'snakemake_cutadapt/reports/{file}_cutadapt.log'
    params:
        adapter_file = 'adapters/adapters.txt'
    shell:
        "cutadapt -a file:{params.adapter_file} -e 0.1 -O 1 --quiet --minimum-length 18 " 
        "--maximum-length 40 -o {output.cutadapt_reads} {input} > {output.log}"


rule bowtie1_alignment:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        aligned_sam = 'snakemake_bowtie/aligned/{file}_bowtie_aligned.sam',
        non_aligned = 'snakemake_bowtie/not_aligned/{file}_bowtie_non_aligned.fastq',
        log = 'snakemake_bowtie/reports/{file}_bowtie_aligned.log'
    params:
        index = 'indexes/bowtie_index/hg_38_index/hg_38'
    shell:
        "bowtie -n 1 -l 10 --best --strata -m 100 -k 1 -x {params.index} {input.reads} "
        "--un {output.non_aligned} -S {output.aligned_sam} 2> {output.log}" 

#bowtie -n 0 -l 32 --norc --best --strata -m 1 (From the previous tries)

Overwriting snakefile_bowtie1_different_parameters


In [199]:
%%writefile snakefile_bowtie1_stringent_parameters

## Testing Alignment with bowtie with parameters found in 
# https://www.biostars.org/p/360709/
# Changing the -m to 1 and -n to 0 to be more stringent
# --> leads to much more stringent alignment, and it filters out much more reads (around 43% vs 1% due to low m)


READS = [file.stem for file in Path('SUBSET_DATA/').iterdir() if file.suffix == '.fastq']

rule all:
    input:
        expand('snakemake_bowtie_stringent/aligned/{file}_bowtie_aligned.sam', file = READS)
        
    
rule remove_umi:
    input:
        raw_reads = 'SUBSET_DATA/{file}.fastq'
    output:
        umi_reads = 'snakemake_umi/umi_trimmed/{file}_umi.fastq',
        log_file = 'snakemake_umi/reports/{file}_umi.log'
    shell:
        "umi_tools extract --extract-method=regex --bc-pattern='(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA)' " 
        "-I {input.raw_reads} -S {output.umi_reads} -L {output.log_file}" 
        #FOR REMOVING 3' ADAPTER: '(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA).*(?P<discard_2>TGGAATTCT.+)' 
        
rule cutadapt:
    input:
        'snakemake_umi/umi_trimmed/{file}_umi.fastq'
    output:
        cutadapt_reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq',
        log = 'snakemake_cutadapt/reports/{file}_cutadapt.log'
    params:
        adapter_file = 'adapters/adapters.txt'
    shell:
        "cutadapt -a file:{params.adapter_file} -e 0.1 -O 1 --quiet --minimum-length 18 " 
        "--maximum-length 40 -o {output.cutadapt_reads} {input} > {output.log}"


rule bowtie1_alignment:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        aligned_sam = 'snakemake_bowtie_stringent/aligned/{file}_bowtie_aligned.sam',
        non_aligned = 'snakemake_bowtie_stringent/not_aligned/{file}_bowtie_non_aligned.fastq',
        log = 'snakemake_bowtie_stringent/reports/{file}_bowtie_aligned.log'
    params:
        index = 'indexes/bowtie_index/hg_38_index/hg_38'
    shell:
        "bowtie -n 0 -l 10 --best --strata -m 1 -k 1 -x {params.index} {input.reads} "
        "--un {output.non_aligned} -S {output.aligned_sam} 2> {output.log}" 



Overwriting snakefile_bowtie1_stringent_parameters


In [203]:
%%writefile snakefile_bowtie1_miRNA

# Testing the parameters that align to the mirBASE dataset

READS = [file.stem for file in Path('SUBSET_DATA/').iterdir() if file.suffix == '.fastq']

rule all:
    input:
        expand('snakemake_bowtie_mirbase/aligned/{file}_bowtie_aligned.sam', file = READS)
        
    
rule remove_umi:
    input:
        raw_reads = 'SUBSET_DATA/{file}.fastq'
    output:
        umi_reads = 'snakemake_umi/umi_trimmed/{file}_umi.fastq',
        log_file = 'snakemake_umi/reports/{file}_umi.log'
    shell:
        "umi_tools extract --extract-method=regex --bc-pattern='(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA)' " 
        "-I {input.raw_reads} -S {output.umi_reads} -L {output.log_file}" 
        #FOR REMOVING 3' ADAPTER: '(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA).*(?P<discard_2>TGGAATTCT.+)' 
        
rule cutadapt:
    input:
        'snakemake_umi/umi_trimmed/{file}_umi.fastq'
    output:
        cutadapt_reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq',
        log = 'snakemake_cutadapt/reports/{file}_cutadapt.log'
    params:
        adapter_file = 'adapters/adapters.txt'
    shell:
        "cutadapt -a file:{params.adapter_file} -e 0.1 -O 1 --quiet --minimum-length 18 " 
        "--maximum-length 40 -o {output.cutadapt_reads} {input} > {output.log}"


rule bowtie1_alignment:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        aligned_sam = 'snakemake_bowtie_mirbase/aligned/{file}_bowtie_aligned.sam',
        non_aligned = 'snakemake_bowtie_mirbase/not_aligned/{file}_bowtie_non_aligned.fastq',
        log = 'snakemake_bowtie_mirbase/reports/{file}_bowtie_aligned.log'
    params:
        index = 'indexes/bowtie_index/mirbase_all_index/mirbase_all'
    shell:
        "bowtie -n 3 -l 10 --best --strata -m 100 -k 1 -x {params.index} {input.reads} "
        "--un {output.non_aligned} -S {output.aligned_sam} 2> {output.log}" 


Overwriting snakefile_bowtie1_miRNA


In [219]:
%%writefile snakefile_bowtie2_ziemann

# File that tries bowtie2.vsl from Zieman article in RNA. This is supposed to be the best aligner! 
# BUT with --end-to-end isntead of --local mode --> this does NOT allow soft clipping!! 

READS = [file.stem for file in Path('SUBSET_DATA/').iterdir() if file.suffix == '.fastq']

rule all:
    input:
        expand('snakemake_bowtie2/{file}_aligned_bowtie2.bam', file = READS),
        'snakemake_featurecounts/featurecounts_genome_clean.csv'
        
rule remove_umi:
    input:
        raw_reads = 'SUBSET_DATA/{file}.fastq'
    output:
        umi_reads = 'snakemake_umi/umi_trimmed/{file}_umi.fastq',
        log_file = 'snakemake_umi/reports/{file}_umi.log'
    shell:
        "umi_tools extract --extract-method=regex --bc-pattern='(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA)' " 
        "-I {input.raw_reads} -S {output.umi_reads} -L {output.log_file}" 
        #FOR REMOVING 3' ADAPTER: '(?P<discard_1>.*)(?P<umi_1>[ACT]{{8}}CA).*(?P<discard_2>TGGAATTCT.+)' 
        
rule cutadapt:
    input:
        'snakemake_umi/umi_trimmed/{file}_umi.fastq'
    output:
        cutadapt_reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq',
        log = 'snakemake_cutadapt/reports/{file}_cutadapt.log'
    params:
        adapter_file = 'adapters/adapters.txt'
    shell:
        "cutadapt -a file:{params.adapter_file} -e 0.1 -O 1 --quiet --minimum-length 18 " 
        "--maximum-length 40 -o {output.cutadapt_reads} {input} > {output.log}"

rule bowtie2_genome_alignment:
    input:
        reads = 'snakemake_cutadapt/{file}_umi_cutadapt.fastq'
    output:
        aligned = 'snakemake_bowtie2/{file}_aligned_bowtie2.bam',
        log = 'snakemake_bowtie2/reports/{file}.log'
    params:
        index = 'indexes/bowtie2_index/genome/GRCh38'
    shell:
        "bowtie2 -k 100 --end-to-end --very-sensitive -x {params.index} -U {input.reads} 2> {output.log} | " 
        "samtools view -b - | samtools sort -o {output.aligned} - && samtools index {output.aligned}"
        
        
rule count_reads_genome_featurecounts:
    input:
        gtf_file = 'mirbase/hsa.gff3'
    params:
        reads_list = 'snakemake_bowtie2/*bam'
    output:
        counts = 'snakemake_featurecounts/featurecounts_genome.txt' 
    shell:
        "featureCounts -t miRNA -g ID -f -O -a {input.gtf_file} -o {output.counts} {params.reads_list}"
        
#featureCounts -t miRNA -g ID -f -s 1 -O -T 16 -a /smallRNA.gtf -o /output_dir n.bam 2>featureCounts.log
#USE -s 1 or not??????? 


# clean columns in featurecounts matrix 
rule clean_featurecounts:
    input:
        'snakemake_featurecounts/featurecounts_genome.txt'
    output:
        'snakemake_featurecounts/featurecounts_genome_clean.csv'
    script:
        'scripts/clean_featurecounts.py'
        


Overwriting snakefile_bowtie2_ziemann
